# Wordle en Español

**Autor:** Pedro Pérez Aguinaga

Introduce palabras de 5 letras y descubre la palabra oculta.

## Importación de librerías

Se importan las bibliotecas necesarias para el proyecto: TensorFlow/Keras para el modelo de red neuronal, NumPy y pandas para manejo de datos, collections.Counter para conteo de letras, requests y json para descargar datasets de palabras en español, re para expresiones regulares y widgets necesarios para Voilà.

In [4]:
"""
Proyecto Final - Inteligencia Artificial
Asistente para Wordle en Español (La palabra del día)
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from collections import Counter
import requests
import json
import re
import random
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output


## Descarga de palabras

Define una única función para descargar, ya que probando a descargar desde fuentes externas como Github tenía errores, además de que no he encontrado una API oficial de la RAE ni la API de donde la propia página de 'La palabra del día' saca sus palabras. También incluye una función para filtrar palabras válidas para Wordle: exactamente 5 letras, sin tildes, sin caracteres especiales y con la letra Ñ.

In [5]:
def descargar_palabras_api():
    """
    Descarga palabras de la API de palabras aleatorias españolas
    """
    # Esta es una fuente más limitada pero funcional
    url = "https://raw.githubusercontent.com/words/an-array-of-spanish-words/master/index.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        palabras = json.loads(response.text)
        print(f"Descargadas {len(palabras)} palabras de API")
        return palabras
    except:
        raise Exception("No se pudieron descargar las palabras de ninguna fuente")

def filtrar_palabras_wordle(palabras):
    """
    Filtra palabras para Wordle: 5 letras, sin tildes, sin caracteres especiales
    """
    palabras_validas = []
    
    for palabra in palabras:
        palabra = palabra.strip().upper()
        
        # Debe tener exactamente 5 letras
        if len(palabra) != 5:
            continue
        
        # Solo letras (sin números ni caracteres especiales)
        if not palabra.isalpha():
            continue
        
        # Sin tildes
        if any(c in palabra for c in 'ÁÉÍÓÚÜ'):
            continue
        
        palabras_validas.append(palabra)
    
    # Eliminar duplicados
    palabras_validas = list(set(palabras_validas))
    
    print(f"Palabras válidas para Wordle: {len(palabras_validas)}")
    return sorted(palabras_validas)

## Procesamiento y análisis de letras

Contiene funciones para analizar la frecuencia de cada letra por posición en las palabras válidas y para crear un vector de características por palabra que incluye frecuencia de letras, score por posición, diversidad de letras y score de frecuencia global.

In [6]:
def crear_estadisticas_letras(palabras):
    """
    Analiza frecuencia de letras por posición
    """
    estadisticas = {i: Counter() for i in range(5)}
    frecuencia_global = Counter()
    
    for palabra in palabras:
        for i, letra in enumerate(palabra):
            estadisticas[i][letra] += 1
            frecuencia_global[letra] += 1
    
    return estadisticas, frecuencia_global

def crear_features_palabra(palabra, estadisticas, frecuencia_global):
    """
    Crea vector de características para una palabra
    Features:
    - 26 valores: frecuencia de cada letra del alfabeto
    - 5 valores: score de cada posición
    - 1 valor: diversidad de letras
    - 1 valor: score total de frecuencia
    """
    alfabeto = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    features = []
    
    # Frecuencia de cada letra en la palabra
    for letra in alfabeto:
        features.append(palabra.count(letra))
    
    # Score de posición para cada letra
    for i, letra in enumerate(palabra):
        score_pos = estadisticas[i].get(letra, 0) / len(estadisticas[i])
        features.append(score_pos)
    
    # Diversidad de letras (penaliza letras repetidas)
    diversidad = len(set(palabra)) / 5.0
    features.append(diversidad)
    
    # Score de frecuencia global
    score_freq = sum(frecuencia_global.get(letra, 0) for letra in palabra)
    score_freq_norm = score_freq / sum(frecuencia_global.values())
    features.append(score_freq_norm)
    
    return np.array(features, dtype=np.float32)

## Creación del modelo de red neuronal

Define la arquitectura de la red neuronal para clasificar palabras según su probabilidad de ser la respuesta correcta, utilizando capas densas y dropout para regularización.

In [7]:
def crear_modelo():
    """
    Red neuronal para scoring de palabras
    """
    modelo = keras.Sequential([
        layers.Input(shape=(33,)),  # 26 + 5 + 1 + 1
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Score de 0 a 1
    ])
    
    modelo.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return modelo

def generar_datos_entrenamiento(palabras, estadisticas, frecuencia_global):
    """
    Genera datos de entrenamiento basados en características lingüísticas
    """
    X = []
    y = []
    
    # Calculamos scores para todas las palabras
    scores = []
    for palabra in palabras:
        # Score basado en frecuencia y diversidad
        score_freq = sum(frecuencia_global.get(letra, 0) for letra in palabra)
        diversidad = len(set(palabra)) / 5.0
        score = (score_freq / sum(frecuencia_global.values())) * diversidad
        scores.append(score)
    
    # Normalizamos scores
    max_score = max(scores)
    min_score = min(scores)
    
    for palabra, score in zip(palabras, scores):
        features = crear_features_palabra(palabra, estadisticas, frecuencia_global)
        X.append(features)
        # Normalizar score a [0, 1]
        y.append((score - min_score) / (max_score - min_score))
    
    return np.array(X), np.array(y)

## Motor de sugerencias

Implementa la clase principal del asistente que gestiona el estado del juego, filtra palabras candidatas según el feedback recibido y genera sugerencias basadas en probabilidad o en maximizar información.

In [8]:
class WordleAssistant:
    def __init__(self, palabras, modelo, estadisticas, frecuencia_global):
        self.palabras = palabras
        self.modelo = modelo
        self.estadisticas = estadisticas
        self.frecuencia_global = frecuencia_global
        self.letras_verdes = {}  # {posicion: letra}
        self.letras_amarillas = set()  # letras que están pero en otra posición
        self.letras_amarillas_pos = {}  # {letra: [posiciones_donde_NO_está]}
        self.letras_negras = set()
    
    def actualizar_feedback(self, palabra, feedback):
        """
        Actualiza el estado con el feedback del intento
        palabra: str, la palabra intentada
        feedback: str, 5 caracteres con 'V'(verde), 'A'(amarillo), 'N'(negro)
        Ejemplo: "VNANV" significa verde-negro-amarillo-negro-verde
        """
        palabra = palabra.upper()
        feedback = feedback.upper()
        
        for i, (letra, color) in enumerate(zip(palabra, feedback)):
            if color == 'V':
                self.letras_verdes[i] = letra
                self.letras_amarillas.discard(letra)  # Ya no es amarilla
            elif color == 'A':
                self.letras_amarillas.add(letra)
                if letra not in self.letras_amarillas_pos:
                    self.letras_amarillas_pos[letra] = []
                self.letras_amarillas_pos[letra].append(i)
            elif color == 'N':
                # Solo marcar como negra si no está verde o amarilla
                if letra not in self.letras_verdes.values() and letra not in self.letras_amarillas:
                    self.letras_negras.add(letra)
    
    def filtrar_candidatas(self):
        """
        Filtra palabras según restricciones actuales
        """
        candidatas = []
        
        for palabra in self.palabras:
            valida = True
            
            # Verificar letras verdes (deben estar en su posición)
            for pos, letra in self.letras_verdes.items():
                if palabra[pos] != letra:
                    valida = False
                    break
            
            if not valida:
                continue
            
            # Verificar letras amarillas (deben estar pero no en ciertas posiciones)
            for letra in self.letras_amarillas:
                if letra not in palabra:
                    valida = False
                    break
                # Verificar que no esté en posiciones descartadas
                if letra in self.letras_amarillas_pos:
                    for pos_no in self.letras_amarillas_pos[letra]:
                        if palabra[pos_no] == letra:
                            valida = False
                            break
            
            if not valida:
                continue
            
            # Verificar letras negras (no deben estar)
            for letra in self.letras_negras:
                if letra in palabra:
                    valida = False
                    break
            
            if valida:
                candidatas.append(palabra)
        
        return candidatas
    
    def sugerir_palabra_probable(self, top_n=5):
        """
        Sugiere las palabras más probables de ser la respuesta
        """
        candidatas = self.filtrar_candidatas()
        
        if not candidatas:
            return []
        
        # Crear features y predecir
        features = np.array([
            crear_features_palabra(p, self.estadisticas, self.frecuencia_global)
            for p in candidatas
        ])
        
        scores = self.modelo.predict(features, verbose=0).flatten()
        
        # Ordenar por score
        indices_ordenados = np.argsort(scores)[::-1]
        
        return [(candidatas[i], float(scores[i])) for i in indices_ordenados[:top_n]]
    
    def sugerir_palabra_exploradora(self, top_n=5):
        """
        Sugiere palabras que maximicen la información (más letras diferentes)
        """
        candidatas = self.filtrar_candidatas()
        
        if not candidatas:
            return []
        
        # Calcular score de exploración
        scores_exploracion = []
        for palabra in candidatas:
            # Contar letras únicas no descubiertas
            letras_conocidas = set(self.letras_verdes.values()) | self.letras_amarillas | self.letras_negras
            letras_nuevas = set(palabra) - letras_conocidas
            # Score: cantidad de letras nuevas * frecuencia promedio
            score = len(letras_nuevas) * sum(self.frecuencia_global.get(l, 0) for l in letras_nuevas)
            scores_exploracion.append(score)
        
        # Ordenar por score de exploración
        indices_ordenados = np.argsort(scores_exploracion)[::-1]
        
        return [(candidatas[i], float(scores_exploracion[i])) for i in indices_ordenados[:top_n]]
    
    def reset(self):
        """Reinicia el asistente para una nueva partida"""
        self.letras_verdes = {}
        self.letras_amarillas = set()
        self.letras_amarillas_pos = {}
        self.letras_negras = set()

## Interfaz

Proporciona funciones para procesar entradas del usuario en formato PALABRA/COLORES y un modo interactivo que permite jugar múltiples partidas con el asistente.

In [9]:
def crear_interfaz(asistente):
    # Estilos CSS
    display(HTML("""
    <style>
        .wordle-title {
            text-align: center;
            color: #2c3e50;
            font-size: 32px;
            font-weight: bold;
            margin: 20px 0;
        }
        .wordle-subtitle {
            text-align: center;
            color: #7f8c8d;
            font-size: 16px;
            margin-bottom: 30px;
        }
        .resultado-box {
            background: #E09116;
            border-left: 4px solid #4CAF50;
            padding: 15px;
            margin: 10px 0;
            border-radius: 5px;
        }
        .candidatas-count {
            font-size: 20px;
        }
        .palabra-sugerida {
            background: #2c3e50;
            padding: 10px;
            margin: 5px 0;
            border-radius: 5px;
            font-family: monospace;
            font-size: 16px;
        }
    </style>
    <div class="wordle-title">Asistente Wordle Español</div>
    <div class="wordle-subtitle">Proyecto de Inteligencia Artificial - La Palabra del Día</div>
    """))
    
    # Widgets
    palabra_input = widgets.Text(
        value='',
        placeholder='Ej: CANTO',
        description='Palabra:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px')
    )
    
    colores_input = widgets.Text(
        value='',
        placeholder='Ej: VAANN',
        description='Colores:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px')
    )
    
    enviar_btn = widgets.Button(
        description='Analizar',
        button_style='primary',
        #layout=widgets.Layout(width='100%')
    )
    
    reset_btn = widgets.Button(
        description='Nueva Partida',
        button_style='warning',
        #layout=widgets.Layout(width='100%')
    )
    
    output = widgets.Output()
    
    # Instrucciones
    instrucciones = widgets.HTML("""
    <div style='background: #2c3e50; padding: 15px; border-radius: 5px; margin: 10px 0;'>
        <b>📖 Instrucciones:</b><br>
        1. Introduce la palabra que probaste en Wordle<br>
        2. Introduce los colores: <b>V</b>=Verde, <b>A</b>=Amarillo, <b>N</b>=Negro<br>
        3. Haz clic en "Analizar" para obtener sugerencias<br><br>
        <i>Ejemplo: Si probaste CANTO y obtuviste 🟩⬛🟨⬛⬛⬛, escribe VAANN</i>
    </div>
    """)
    
    def procesar_intento(b):
        with output:
            clear_output()
            palabra = palabra_input.value.strip().upper()
            colores = colores_input.value.strip().upper()
            
            # Validaciones
            if len(palabra) != 5:
                print("❌ Error: La palabra debe tener 5 letras")
                return
            if len(colores) != 5:
                print("❌ Error: Debes proporcionar 5 colores")
                return
            if not all(c in 'VAN' for c in colores):
                print("❌ Error: Solo usa V, A o N para los colores")
                return
            
            # Actualizar y obtener sugerencias
            asistente.actualizar_feedback(palabra, colores)
            candidatas = asistente.filtrar_candidatas()
            
            display(HTML(f"""
            <div class="resultado-box">
                <h3>Intento procesado: {palabra} → {colores}</h3>
                <p class="candidatas-count">Candidatas restantes: {len(candidatas)}</p>
            </div>
            """))
            
            if len(candidatas) == 0:
                print("⚠️ No quedan candidatas. Verifica los colores ingresados.")
                return
            
            # Top palabras probables
            print("\n🏆 TOP 5 PALABRAS MÁS PROBABLES:")
            probables = asistente.sugerir_palabra_probable(top_n=5)
            for i, (pal, score) in enumerate(probables, 1):
                display(HTML(f'<div class="palabra-sugerida">{i}. <b>{pal}</b> (confianza: {score:.4f})</div>'))
            
            # Palabras exploradoras
            print("\n🔍 TOP 5 PALABRAS QUE APORTAN MÁS INFORMACIÓN:")
            exploradoras = asistente.sugerir_palabra_exploradora(top_n=5)
            for i, (pal, score) in enumerate(exploradoras, 1):
                display(HTML(f'<div class="palabra-sugerida">{i}. <b>{pal}</b> (info: {score:.1f})</div>'))
            
            # Estado actual
            if asistente.letras_verdes or asistente.letras_amarillas or asistente.letras_negras:
                print("\n📌 ESTADO ACTUAL:")
                if asistente.letras_verdes:
                    print(f"   🟩 Verdes: {asistente.letras_verdes}")
                if asistente.letras_amarillas:
                    print(f"   🟨 Amarillas: {asistente.letras_amarillas}")
                if asistente.letras_negras:
                    print(f"   ⬛ Negras: {asistente.letras_negras}")
            
            # Limpiar inputs
            palabra_input.value = ''
            colores_input.value = ''
    
    def resetear(b):
        with output:
            clear_output()
            asistente.reset()
            palabra_input.value = ''
            colores_input.value = ''
            print("🔄 Asistente reiniciado. ¡Nueva partida iniciada!")
    
    enviar_btn.on_click(procesar_intento)
    reset_btn.on_click(resetear)
    
    # Layout
    input_box = widgets.VBox(
        [
            widgets.HBox([palabra_input, colores_input]),
            enviar_btn,
            reset_btn
        ],
        layout=widgets.Layout(
            width='90%',
            gap='15px'
        )
    )
    
    contenedor = widgets.HBox(
        [input_box],
        layout=widgets.Layout(justify_content='center')
    )

    
    display(instrucciones)
    display(input_box)
    display(output)

## Ejecución principal

Contiene la función main() que orquesta todo el flujo del programa: descarga de palabras, entrenamiento del modelo, inicialización del asistente y gestión del modo interactivo.

In [10]:
def main():
    
    # 1. Descargar y preparar palabras
    print("Inicializando asistente...")
    palabras_raw = descargar_palabras_api()
    palabras = filtrar_palabras_wordle(palabras_raw)
    print(f" {len(palabras)} palabras cargadas")
    
    # 2. Crear estadísticas
    print("\n2. Analizando frecuencias de letras...")
    estadisticas, frecuencia_global = crear_estadisticas_letras(palabras)
    print(" Estadísticas calculadas")
    print(f"   Letras más comunes: {frecuencia_global.most_common(5)}")
    
    # 3. Preparar datos de entrenamiento
    print("\n3. Generando datos de entrenamiento...")
    X, y = generar_datos_entrenamiento(palabras, estadisticas, frecuencia_global)
    
    # 4. Entrenar modelo
    print("\n4. Entrenando modelo de red neuronal...")
    modelo = crear_modelo()
    split_idx = int(0.8 * len(X))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    
    print("Entrenando modelo...")
    modelo.fit(X_train, y_train, validation_data=(X_val, y_val), 
               epochs=20, batch_size=32, verbose=1)
    print("Modelo entrenado")
    
    # 5. Crear asistente
    print("\n5. Inicializando asistente...")
    asistente = WordleAssistant(palabras, modelo, estadisticas, frecuencia_global)
    
    print("\n" + "="*70)
    print("ASISTENTE LISTO")
    print("="*70)
    
    # Guardar modelo y datos
    print("\n6. Guardando modelo y datos...")
    modelo.save('wordle_assistant_model.keras')
    print("   Modelo guardado como 'wordle_assistant_model.keras'")
    
    with open('palabras_wordle.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(palabras))
    print("   Palabras guardadas en 'palabras_wordle.txt'")
    
    return asistente, modelo, palabras, estadisticas, frecuencia_global

# Ejecutar
# Inicializar el asistente
asistente, modelo, palabras, estadisticas, frecuencia_global = main()

# Crear interfaz interactiva
crear_interfaz(asistente)

Inicializando asistente...
Descargadas 636598 palabras de API
Palabras válidas para Wordle: 10836
 10836 palabras cargadas

2. Analizando frecuencias de letras...
 Estadísticas calculadas
   Letras más comunes: [('A', 8814), ('E', 5378), ('O', 5337), ('S', 3844), ('R', 3666)]

3. Generando datos de entrenamiento...

4. Entrenando modelo de red neuronal...
Entrenando modelo...
Epoch 1/20


2025-12-23 21:59:21.333089: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


271/271 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 8.8006e-04 - loss: 1.2743 - val_accuracy: 9.2251e-04 - val_loss: 0.6885
Epoch 2/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0011 - loss: 0.6893 - val_accuracy: 4.6125e-04 - val_loss: 0.6865
Epoch 3/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 4.5770e-04 - loss: 0.6818 - val_accuracy: 4.6125e-04 - val_loss: 0.6871
Epoch 4/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 8.2758e-04 - loss: 0.6810 - val_accuracy: 4.6125e-04 - val_loss: 0.6859
Epoch 5/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 5.6948e-04 - loss: 0.6775 - val_accuracy: 9.2251e-04 - val_loss: 0.6849
Epoch 6/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0018 - loss: 0.6752 - val_accuracy: 4.6125e-04 - val_loss: 0.6809
Epoch 7/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0012 - loss: 0.6741 - val_accuracy: 4.6125e-04 - val_loss: 0.6792
Epoch 8/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 7.4509e

HTML(value='\n    <div style=\'background: #2c3e50; padding: 15px; border-radius: 5px; margin: 10px 0;\'>\n   …

Output()